### Branje podatkov in metainformacij o EEG meritvi

In [54]:
import numpy as np
from sklearn.decomposition import FastICA
import matplotlib.pyplot as plt
import mne
import pandas as pd
from plotly import tools
import plotly.graph_objects as go
import re
from IPython.display import Image
import scipy as sc
from scipy import signal
import math

SUB_NUM = 2
EXP_NUM = 4
DATA_ROOT = "/home/erik/Documents/Projects/Faculty/human_computer_communication/seminarska_2/data/files/"
IMAGE_SAVE_PATH = "/home/erik/Documents/Projects/Faculty/human_computer_communication/seminarska_2/images/"

SUBJECT  = f"S00{SUB_NUM}/S00{SUB_NUM}R0{EXP_NUM}.edf"
FILE = DATA_ROOT+SUBJECT
print(f"Reading file: {FILE}")
data = mne.io.read_raw_edf(FILE,stim_channel="anotations")
# data.info
ch_names = data.ch_names
np_data = data.get_data()
sampling_freq = data.info["sfreq"]
no_channesl = np_data.shape[0]
no_samples = np_data.shape[1]
recorded_time = no_samples/sampling_freq

print(f"No channels: {no_channesl}")


Reading file: /home/erik/Documents/Projects/Faculty/human_computer_communication/seminarska_2/data/files/S002/S002R04.edf
Extracting EDF parameters from /home/erik/Documents/Projects/Faculty/human_computer_communication/seminarska_2/data/files/S002/S002R04.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
No channels: 64


### Definicija funkcij za izrisovanje grafov
Za izrisovanje grafov uporabljamo knjižnico Plotly. Grafi se shranjujejo v mapo, ki je definirana v spremenljivki IMAGE_SAVE_PATH.

In [55]:
def plot_eeg(data,ch_names,filename):
    
    no_channesl = data.shape[0]
    no_samples = data.shape[1]
    
    fig = go.Figure()
    increment = 0    
    y_increments = []
    for i in range(no_channesl):
        fig.add_trace(go.Scatter(x=list(range(no_samples)),y=data[i] + increment,mode="lines",name=f"{ch_names[i]}"))
        y_increments.append(increment)
        increment += 200E-6


    fig.update_layout(showlegend=False)
    fig.update_layout(
        xaxis = dict(
            tickmode = 'array',
            tickvals = [x* sampling_freq for x in range(int(np.ceil(recorded_time))) if x % 5 == 0],
            ticktext = [f"{x}s" for x in range(int(np.ceil(recorded_time))) if x % 5 == 0],
            gridcolor = "rgb(159, 197, 232)",        
            zeroline = False
        )
    )
    fig.update_layout(
        yaxis = dict(
            tickmode = 'array',
            tickvals = y_increments,
            ticktext = ch_names,
            gridcolor = "rgb(159, 197, 232)",
            zeroline = False,
            showgrid = False,
            tickfont=dict(family='Rockwell', color='crimson', size=5)

        )
    )
    fig.write_image(filename)

def plot_ica_components(components,filename):
    fig = go.Figure()
    increment = 0
    dif = np.amax(components)
    y_increments = []
    for i in range(no_channesl):
        fig.add_trace(go.Scatter(x=list(range(no_samples)),y=components.T[i] + increment,mode="lines"))
        y_increments.append(increment)
        increment += dif


    fig.update_layout(showlegend=False)
    fig.update_layout(
        xaxis = dict(
            tickmode = 'array',
            tickvals = [x* sampling_freq for x in range(int(np.ceil(recorded_time))) if x % 5 == 0],
            ticktext = [f"{x}s" for x in range(int(np.ceil(recorded_time))) if x % 5 == 0],
            gridcolor = "rgb(159, 197, 232)",        
            zeroline = False
        )
    )
    fig.update_layout(
        yaxis = dict(
            tickmode = 'array',
            tickvals = y_increments,
            ticktext = [f"ICA{x}" for x in range(no_channesl)],
            gridcolor = "rgb(159, 197, 232)",
            zeroline = False,
            showgrid = False,
            tickfont=dict(family='Rockwell', color='crimson', size=5)

        )
    )

    fig.write_image(filename)

### Izris grafa vhodnih podatkov

In [56]:
plot_eeg(np_data,ch_names,"images/test.svg")

### Izračun ANK 

In [57]:
filt_raw = data.copy()
filt_raw.load_data().filter(l_freq=1., h_freq=None)
np_filtered = filt_raw.get_data()


ica = FastICA(n_components=64,algorithm="parallel",whiten=True,max_iter=1000,tol=0.0001)
ica.fit_transform(np_filtered.T)
components = ica.transform(np_filtered.T)


Reading 0 ... 19679  =      0.000 ...   122.994 secs...
Filtering raw data in 1 contiguous segment
Setting up high-pass filter at 1 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal highpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Filter length: 529 samples (3.306 sec)



(19680, 64)

### Izris grafa signala v prostoru komponent

In [58]:
plot_ica_components(components,"images/test_ica.svg")

### Izbor ANK komponent ki so rezultat mežikanja
za signale posameznih komponent preverimo kako korelirajo z signali frontalnih elektrod Fp1., Fp2. in Fpz. .Pri navedenih frontalnih elektrodah je intenziteta artefaktov namreč najvelja.

In [59]:
picks = mne.pick_channels(ch_names,include=["Fp1.","Fp2.","Fpz."])
eye_channels = data.get_data(picks=picks)

b, a = signal.butter(5, 0.005)
averaged_eye = []
for sig in eye_channels:
    y = signal.filtfilt(b, a, sig)
#     y = smooth(sig,3)
#     y = signal.savgol_filter(sig,11,3)    
    averaged_eye.append(y)  

averaged_eye = np.array(averaged_eye)


(3, 19680)


Kot kompenente, ki jih želimo izključiti vzamemo dve komponenti, ki najbolj pozitivno korelirata s signali frontalnih elektrod ter dve komponenti, ki najbolj negativno korelirata s signalom frontalnih elektrod

In [60]:
b, a = signal.butter(5, 0.005)
pearsons_per_ica = []
i = 0
for ic in components.T:
    n_ica = signal.filtfilt(b, a, ic)
    max_ica = [-math.inf,None]
    for j in range(eye_channels.shape[0]):
        r,p = sc.stats.pearsonr(n_ica,averaged_eye[j])
        raux = r
        if raux < 0:
            raux = -raux
        
        if(raux>max_ica[0]):
            max_ica[0] = r
            max_ica[1] = i
    pearsons_per_ica.append(max_ica)
    i+= 1

pearsons_per_ica = sorted(pearsons_per_ica, key=lambda x: x[0])
to_exclude = pearsons_per_ica[:2] +  pearsons_per_ica[-2:]
to_exclude = [x[1] for x in to_exclude]
to_exclude


[10, 26, 2, 47]

Izbrane komponente nadomestimo z ničelnim vektorjeml in rekonstruiramo signal brez artefaktov.

In [61]:
components.shape
comp_copy = np.copy(components)
comp_copy[:, to_exclude] = 0
restored = ica.inverse_transform(comp_copy)

(19680, 64)

### Izris grafa rekonstruiranega signala

In [62]:
plot_eeg(restored.T,ch_names,"images/test_restored.svg")

In [63]:
# plot_eeg(averaged_eye,["Fp1.","Fp2.","Fpz."],"images/test_sub2.svg")
# plot_eeg(eye_channels,["Fp1.","Fp2.","Fpz."],"images/test_sub.svg")
